# Homework 3 - What movie to watch tonight?

## 1. Data collection

The data collection task is divided into **three** subtasks.
* Get the list of movies
* Crawl Wikipedia
* Parse downloaded pages

#### Import of librabries

In [ ]:
import requests
import pandas as pd
import time
import os
from bs4 import Tag, NavigableString, BeautifulSoup
import time
import csv
import itertools
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#### Get the list of movies 

In [ ]:
big_page = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html'
big_page_response = requests.get(big_page)

In [ ]:
big_page_soup = BeautifulSoup(big_page_response.text, 'html.parser')

#### Crawl Wikipedia

In [ ]:
List_url=[]

In [ ]:
counter = 0
for i in big_page_soup.find_all('tr')[1:]:
    url = i.find_all('a')[0].get("href") # get the url in the second column of each row starting from the sec. row
    page_response= requests.get(url)  # reponse to the request
    
    if page_response.status_code == 200:  # 200 == means everthing is ok
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            page_soup = BeautifulSoup(page_response.text, 'html.parser')
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1
    elif page_response.status_code == 429: # 429 == means that your doing too many requaets
        print('we must wait...')
        time.sleep(600) # wait 10 mins 
        page_soup = BeautifulSoup(page_response.text, 'html.parser')
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1
    elif counter%25==0:  # every 25 files, wait 10 seconds
        time.sleep(10)
        page_soup = BeautifulSoup(page_response.text, 'html.parser')
        with open(os.path.join('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files1',"movie"+str(counter+1)+".html"), "w") as file:
            file.write(str(page_soup))  # writes the html page
            List_url.append(url) # add url to the list L
        counter += 1


    print("parsed movie " + str(counter))

#### Parse Downloaded pages

In [ ]:
ps = PorterStemmer()
stop = set(stopwords.words('english'))
Vocabulary = {}

In [ ]:
# The following function is to clean the rows of the intobox
def clean_col(string):
    clean1 = string.strip('\n') # remove all escape characters
    pattern = r'\[.*?\]'  # remove all square brakets and content
    clean2 = re.sub(pattern, '', clean1)
    return(clean2)

# The following function is to extract info from the intobox

# This function takes three inputs:
# The first input is the html page - from which the info is going to be extracted.
# The second input is a number associated to the page (This number is an identificator number, 
# thanks to which we will be able to retrieve the url of the html page).
# The third input is the name given to the html page.

def info_tsv(film_soup, movie_number, filename):  
    
    # This is the list of the info we want to take from the infobox
    infb = ['Directed by', 'Produced by', 'Written by', 'Screenplay by',
           'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by',
           'Cinematography', 'Edited by', 'Production company', 
            'Distributed by', 'Release date', 'Running time', 'Country',
           'Language', 'Budget'] 
    
    # remove all square brakets and content
    pattern = r'\[.*?\]' 
    
    # This list will be a list of list, and each list inside will contain the infobox related to each row, the Into and Plot
    L = []  
        
    # Retrieve the Title   
        
    Initial_Title = film_soup.select("#firstHeading")[0].text
    L.append(['Title', str(Initial_Title)])
    
    # Retrieve the Intro
     
    contents = film_soup.findAll('div', attrs={'id': 'toc'}) 
    if len(contents) > 0:
        Intr = contents[0].fetchPreviousSiblings('p') # give us all the paragraphs from the end of the first section
        # These paragraphs correspond to the Intro
        Intro = []
        if len(Intr) > 0:
            for p in reversed(Intr):  # we order the paragraphs as they should be (same order as in the original html page)
                if len(p.text) > 5:  # If a paragraph has less than five strings we do not consider it to be a paragraph
                    Intro.append(p.text)  # The treshold len(p.text) > 5 help us to remove all empty and meaningless paragraphs
                else:
                    continue
            Intro = ''.join(Intro)
            Final_intro = re.sub(pattern, '', Intro)  # remove all the square brakets and what it contains, because  
                                                      # These brakets are Wikipedia notes/citations
            if len(Final_intro)>20: # If a section, in this case the Intro, has less than 20 strings, we do consider it to be meaningless
                L.append(['Intro', Final_intro.strip()])  # we save the info we've got from the intro
            else:
                L.append(['Intro', 'NA'])  # in case we don't have any info, we put an 'NA'
    
    
    # Retrieve the Plot
    
    plot = []
    
    # Before starting looking for the info in the plot, we created a list of the most used names to indicate the section Plot
    possibles = ['Plot','Synopsis','Plot synopsis','Plot summary', 'Plot_summary', 'Plot_synopsis'
                 'Story','Plotline','The Beginning','Summary',
                'Content','Premise', 'Intro', 'Intro']
    for i in possibles:
        items = film_soup.find(id=i) # se find the section with name == ith element in possibles
        if items is None or len(items) == 0:
            continue
        else:
            # walk through the siblings of the parent (H2) node 
            # until we reach the next H2 node
            for element in items.parent.nextSiblingGenerator():  # we basically take all the paragraphs till the end of the section
                if element.name == "h2":
                    break
                if hasattr(element, "text"):
                    plot.append(element.text)
    plot = "".join(plot)
    # remove all square brakets and content
    Final_plot = re.sub(pattern, '', plot)
    if len(Final_plot) > 20:  # same as before (for the Intro), in case the whole section has less than 20 string, we considered it meaningless
        L.append(['Plot', Final_plot.strip()])  # we save the info found in the Plot
    else:
        L.append(['Plot', 'NA']) # in case we don't have any info, we put an 'NA'
    
    # Retrieve info in the infobox
    
    # we have found the infobox by not only looking for the tables in the page but by section the table with class = 'infobox vevent'  
    if len(film_soup.findAll('table', {'class': 'infobox vevent'}))>0:  
        
        # If the box is not empty we go through it
        if len(film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr'))>0:
            
            # from the infobox the firt row correspond to the Name of the Film
            Title = film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr')[0].text
            
            # If a title contains more than 75 strings we do not consider it to a title, because it could be some additional 
            # messages inserted in the pages
            if len(Title)>75:
                
                L.append(['Name', 'NA']) # in case we don't have any info, we put an 'NA'
            else:
                L.append(['Name', Title]) # we save the Name of the film
            
        else:
            L.append(['Name', 'NA']) # in case we don't have any info, we put an 'NA'
    
    
    # In the following table we will put all the info that we've retreived from the infobox - except the name.
    # The the idea, is to do an intersection between the elements found and the elements we wanted to found
    # for all the missing elements we will put 'NA'
    
    in_table = [] # This list will contain all the info found in the infobox
    
    # check if the table is not empty
    if len(film_soup.findAll('table', {'class': 'infobox vevent'}))>0: 
        
        # check if the row is not empty
        if len(film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr'))>0:
            
            #go through the columns of each row
            for row in film_soup.findAll('table', {'class': 'infobox vevent'})[0].tbody.findAll('tr')[1:]:
                
                # check that the first column is not empty
                first_col = row.findAll('th') 
                if len(first_col)>0:
                    
                    # extract and check the second column
                    second_col = row.findAll('td')
                    if len(second_col)>0:
                        
                        # external rows can contain internal rows. 
                        # extract elements in each row 
                        second_col = []
                        for element in row.findAll('td')[0]:
                            # check if the element in the row has a child
                            if isinstance(element, Tag):
                                second_col.append(element.text) # collect all the info found in this row
                            else:
                                second_col.append(element)
                        second_col = ' '.join(second_col)
                        
                        new_second_col = clean_col(second_col) # clean the output -calling the initial function

                        in_table.append(first_col[0].text) # save the feature of the row ex. 'Directed by'
                        in_table.append(new_second_col)  # save the info corresponding to this feature
                        

            # put 'NA' to all missing values
            for i in infb:
                if i in in_table:
                    index = in_table.index(i)
                    L.append([i, in_table[index +1]])
                else:
                    L.append([i, 'NA']) # in case we don't have any info, we put an 'NA'
    L.append(['Url', List_url[int(movie_number) -1]])  # from the list of the urls we take url of this html page
    return(L) # we return all the info we need to save in the tsv files.

#### Creation of the tsv files

In [ ]:
paths = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files/'

    
for filename in os.listdir(paths):
    if filename not in movies_done:
        if filename.endswith(".html"):
            ff = open(paths + filename, 'r') # open the HTML page
            html = ff.read()
            film = BeautifulSoup(html)

            page_name = filename.rstrip('.html') # extract the name of the file without extension and we will use this as the file id            
            movie_number = re.findall('\d+', filename )
            movie_number = movie_number[0]
            
            
            #save the info into tsv files
            with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'+page_name+'.tsv', 'wt') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                if info_tsv(film,movie_number , filename) is None or len(info_tsv(film, movie_number, filename)) == 0:
                    continue
                else:
                    for col in info_tsv(film, movie_number, filename):
                        tsv_writer.writerow([col[0], col[1]])
            print('tsv created '+ str(len(movies_done)))

            continue

        else:
            continue
    else:
        continue
print('DONE')

## 2. Search Engine

#### Import libraries

In [ ]:
import json
import numpy as np
import pandas as pd
import os
import csv
import itertools
import re
import csv
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from itertools import chain
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity    
import operator
from nltk.corpus import wordnet as wn
import math
from collections import Counter

In [ ]:
ps = PorterStemmer()
attr = ['Intro', 'Plot']
stop = set(stopwords.words('english'))

In [3]:
path = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'

### Create your index!

This part has been done in the previous part, directly while creating the tsv files.

In [ ]:
good_ict = {}
Vocabulary = list()

In [ ]:
attr = ['Plot', 'Intro', 'Directed by', 'Produced by', 'Written by', 'Screenplay by',
           'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by',
           'Cinematography', 'Edited by', 'Production company', 
            'Distributed by',  'Country',
           'Language']

i = 0
for filename in os.listdir(path):  
    if filename.endswith(".tsv"):
        page_name = filename.rstrip('.tsv') # without extension
        movie_number = re.findall('\d+', filename )
        movie_number = movie_number[0]
        words = []
        with open(path+filename) as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row[0] in attr:
                    for word in row[1].strip().split():
                        if word == 'NA':
                            continue
                        else:
                            word_cl1 = re.sub(r'[^\w\s]','',word)
                            word_cl1 = word_cl1.lower()
                            if word_cl1 not in stop:
                                stemmed_word = ps.stem(word_cl1)
                                words.append(stemmed_word)
        words = list(set(words))
        for clean_word in words:
            if clean_word not in Vocabulary:
                Vocabulary.append(clean_word)
            else:
                continue
        for clean_word in words: 
            term_id = Vocabulary.index(clean_word)
            if str(term_id) not in good_dict.keys():
                good_dict[str(term_id)] = [filename]
            else:
                if filename in good_dict[str(term_id)]:
                    continue
                else:
                    good_dict[str(term_id)].append(filename)
print('Done')

### Save the dictionary and vocabulary

In [ ]:
with open('/Users/yves/Desktop/dictionary1.json', 'w') as outfile:
    json.dump(good_dict, outfile)

In [ ]:
with open('/Users/yves/Desktop/Vocabulary.txt', 'w') as f:
    for item in Vocabulary:
        f.write("%s\n" % item)

### Load the Initial dictionary

In [ ]:
f =  open('/Users/yves/Desktop/dictionary1.json', 'r')
good_dict = json.load(f)

## Load Vocabulary

In [ ]:
Vocabulary = open("/Users/yves/Desktop/Vocabulary.txt", "r", 
                       encoding = "utf-8")          
Vocabulary = Vocabulary.read()      
Vocabulary = Vocabulary.split("\n")

##  Execute the query

### load your query

In [44]:
initial_query = input("Enter string:  ")
initial_query = initial_query.split()

Enter string:  love movie


### Selection of the documents containg all the words in the query

In [45]:
# function for stemming 
def stem_word(lst_words):
    clean_lst_words= []
    for word in lst_words:
        word = ps.stem(word)
        clean_lst_words.append(word)
    clean_lst_words = list(set(clean_lst_words))
    return(clean_lst_words)

final_query = []
for word in initial_query:
    for ss in wn.synsets(word):
        final_query.append((ss.lemma_names()[0]))
        
query = stem_word(final_query)

In [46]:
lst = []
for word in query:
    if word in Vocabulary:
        term_id = Vocabulary.index(word)
        docs = good_dict[str(term_id)]
        if docs is None or len(docs) == 0:
            continue
        else:
            for i in docs:
                lst.append(i)
result=[]
for i in lst:
    if lst.count(i) >= len(initial_query)-1:
         result.append(i)
result = list(set(result))

In [50]:
to_df = []
for movie in result:
    element_in_movie = {}
    with open(path+movie) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row[0] == 'Name':
                element_in_movie['Name'] = row[1]
            elif row[0] == 'Intro':
                element_in_movie['Intro'] = row[1]
            elif row[0] == 'Url':
                element_in_movie['Url'] = row[1]
    to_df.append(element_in_movie)
df = pd.DataFrame(to_df)
df = df.reindex(('Name', 'Intro', 'Url'), axis=1)
df.iloc[:5]

,Name,Intro,Url
0,Fired Up!,Fired Up! is a 2009 American teen comedy film ...,https://en.wikipedia.org/wiki/Fired_Up!
1,Going the Distance,Going the Distance is a 2010 American romantic...,https://en.wikipedia.org/wiki/Going_the_Distan...
2,Scott Pilgrim vs. the World,Scott Pilgrim vs. the World is a 2010 action c...,https://en.wikipedia.org/wiki/Scott_Pilgrim_vs...
3,An Invisible Sign,An Invisible Sign is a 2010 American drama fil...,https://en.wikipedia.org/wiki/An_Invisible_Sign
4,Blue Valentine,Blue Valentine is a 2010 American romantic dr...,https://en.wikipedia.org/wiki/Blue_Valentine_(...


## Conjunctive query & Ranking score

### Inverted index


<p>
    TF = (Frequency of the word in the sentence) / (Total number of words in the sentence)
</p>
<p>
    IDF: (Total number of sentences (documents))/(Number of sentences (documents) containing the word)
</p>

In [30]:
attr = ['Intro', 'Plot']  
ps = PorterStemmer()
stop = set(stopwords.words('english'))
lst = list(good_dict.keys())
len_dict = len(os.listdir(path))
New_Dict = {}

for term_id in good_dict.keys():  # for each word in the dictionary
    val_word = list(good_dict[term_id]) # we save the number of documents having that word
    for doc in good_dict[term_id]:  # we go through each document
        file = [] # here we will save the the words contained in the document
        with open(path+doc) as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row[0] in attr:
                    for word in row[1].strip().split():
                        if word == 'NA':
                            continue
                        else:
                            word_cl1 = re.sub(r'[^\w\s]','',word)
                            word_cl1 = word_cl1.lower()
                            if word_cl1 not in stop:
                                stemmed_word = ps.stem(word_cl1)
                                file.append(stemmed_word)
        if len(file) < 1:
            if word in New_Dict.keys():  # add the word to the new dictioary if the word is not already inside and add the new value if the word is already inside
                New_Dict[word].append((doc, 0))
            else:
                New_Dict[word] = [(doc, 0)]
        else:
            len_file = len(file) # the total number of words
            word = Vocabulary[int(term_id)]
            count = file.count(word) # the number of times that word is repeated in the document
            tf = round(count/len_file, 4) # the frequency of the word in the document
            idf = np.log(len_dict/len(val_word)) # the IDF 
            if term_id in New_Dict.keys():  # add the word to the new dictioary if the word is not already inside and add the new value if the word is already inside
                New_Dict[term_id].append((doc, tf*idf))
            else:
                New_Dict[term_id] = [(doc, tf*idf)]

#### save the inverted index

In [ ]:
with open('/Users/yves/Desktop/tfidf.json', 'w') as outfile:
    json.dump(New_Dict, outfile)

#### load the inverted index

In [ ]:
f =  open('/Users/yves/Desktop/partial_DICT_tfidf.json', 'r')
New_Dict = json.load(f)

### Query TF-IDF


In [52]:
def tf_idf_query(query):
    tf_idf = {}
    len_query = len(query)
    len_dict = len(os.listdir(path))
    DF = {}
    for word in query:# its a count funtion
        if word not in DF.keys():
            DF[word] = 1 
        else:
            DF[word]+=1
    
    for word in query:
        
        # TF = (Frequency of the word in the sentence) / (Total number of words in the sentence)
        count = query.count(word)
        freq = count/len_query
        tf = freq
        
        val_word = DF[word]
        idf = idf = np.log(len_dict/val_word)
        if word in Vocabulary:
            term_id = Vocabulary.index(word)
            if word in tf_idf.keys():
                tf_idf[str(term_id)]+= tf*idf
            else:
                tf_idf[str(term_id)] = tf*idf
    return tf_idf

In [53]:
tf_idf_query = tf_idf_query(query) 

### Vectorization, Cosine Similarity and Ranking

In [54]:
result_list = []
for i in range(len(result)):
    small_dict = {}
    for word in New_Dict.keys():
        for files in New_Dict[word]:
            if files[0] == result[i]:
                small_dict[word] = files[1]
    result_list.append(small_dict)

In [55]:
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [56]:
cos_result = {}
for i in range(len(result)):
    cosine = get_cosine(tf_idf_query, result_list[i])
    cos_result[result[i]] = cosine*100
    
cos_result = sorted(cos_result.items(), key=operator.itemgetter(1), reverse=True)

In [57]:
to_df = []
for movie in cos_result:
    element_in_movie = {}
    with open(path+movie[0]) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row[0] == 'Name':
                element_in_movie['Name'] = row[1]
            elif row[0] == 'Intro':
                element_in_movie['Intro'] = row[1]
            elif row[0] == 'Url':
                element_in_movie['Url'] = row[1]
        element_in_movie['Similarity'] = str(movie[1])+' %'
    to_df.append(element_in_movie)
df = pd.DataFrame(to_df)
df = df.reindex(('Name', 'Intro', 'Url', 'Similarity'), axis=1)
df.iloc[:10]

,Name,Intro,Url,Similarity
0,When in Rome,When in Rome is a 2010 American romantic comed...,https://en.wikipedia.org/wiki/When_in_Rome_(20...,6.835857966671183 %
1,Dear John,Dear John is a 2010 American romantic drama-wa...,https://en.wikipedia.org/wiki/Dear_John_(2010_...,5.704633376565299 %
2,I Love You Phillip Morris,I Love You Phillip Morris is a 2009 black come...,https://en.wikipedia.org/wiki/I_Love_You_Phill...,5.264481089373219 %
3,My One and Only,My One and Only is a 2009 comedy-drama film lo...,https://en.wikipedia.org/wiki/My_One_and_Only_...,4.689575381407523 %
4,Never Let Me Go,Never Let Me Go is a 2010 British dystopian ro...,https://en.wikipedia.org/wiki/Never_Let_Me_Go_...,4.209033222443061 %
5,Eat Pray Love,Eat Pray Love is a 2010 American biographical ...,https://en.wikipedia.org/wiki/Eat_Pray_Love,3.918970377109158 %
6,I Can Do Bad All by Myself,I Can Do Bad All by Myself is a 2009 romantic ...,https://en.wikipedia.org/wiki/I_Can_Do_Bad_All...,3.623062757513399 %
7,The Informers,The Informers is a 2008 American ensemble Holl...,https://en.wikipedia.org/wiki/The_Informers_(2...,3.2380426575731334 %
8,The Limits of Control,The Limits of Control is a 2009 American film ...,https://en.wikipedia.org/wiki/The_Limits_of_Co...,3.1846058945767446 %
9,Hurricane Season,Hurricane Season is a 2010 sports drama film d...,https://en.wikipedia.org/wiki/Hurricane_Season...,3.1793814369053632 %


## Define a new score!

### New inverted matrix

In [ ]:
i = []
ps = PorterStemmer()
stop = set(stopwords.words('english'))
len_dict = len(os.listdir(path))
attr = ['Plot', 'Intro']
important = ['Directed by', 'Produced by', 'Written by', 'Screenplay by',
           'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by',
           'Cinematography', 'Edited by', 'Production company', 
            'Distributed by',  'Country',
           'Language']

New_Dict = {}
for term_id in good_dict.keys():
    val_word = list(good_dict[term_id])
    for doc in good_dict[term_id]:

        intro_plot = []
        important1 =[]
        title = []
        with open(path+doc) as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row[0] in attr:
                    for word in row[1].strip().split():
                        if word == 'NA':
                            continue
                        else:
                            word_cl1 = re.sub(r'[^\w\s]','',word)
                            word_cl1 = word_cl1.lower()
                            if word_cl1 not in stop:
                                stemmed_word = ps.stem(word_cl1)
                                intro_plot.append(stemmed_word)
                elif row[0] in important:
                    for word in row[1].strip().split():
                        if word == 'NA':
                            continue
                        else:
                            word_cl1 = re.sub(r'[^\w\s]','',word)
                            word_cl1 = word_cl1.lower()
                            if word_cl1 not in stop:
                                stemmed_word = ps.stem(word_cl1)
                                important1.append(stemmed_word)
                elif row[0] == 'Title':
                    for word in row[1].strip().split():
                        if word == 'NA':
                            continue
                        else:
                            word_cl1 = re.sub(r'[^\w\s]','',word)
                            word_cl1 = word_cl1.lower()
                            if word_cl1 not in stop:
                                stemmed_word = ps.stem(word_cl1)
                                title.append(stemmed_word)


        if len(intro_plot) < 1:
            continue
        else:
            len_file = len(intro_plot) # the total number of words
            word = Vocabulary[int(term_id)]
            count = intro_plot.count(word) # the number of times that word is repeated in the document
            tf_in = count/len_file # the frequency of the word in the document
            idf = np.log(len_dict/len(val_word)) # the IDF 

            tf_title = 0
            if len(title) > 0:
                tf_title = title.count(word)/len(title)
            tf_imp = important1.count(word)/len(important1)
            if tf_title > 0:
                tf = tf_in + tf_imp + (tf_title)**2

                if term_id in New_Dict.keys():  # add the word to the new dictioary if the word is not already inside and add the new value if the word is already inside
                    New_Dict[term_id].append((doc, tf*idf))
                else:
                    New_Dict[term_id] = [(doc, tf*idf)]
            else:
                if tf_imp > 0:
                    tf = tf_in + tf_imp

                    if term_id in New_Dict.keys():  # add the word to the new dictioary if the word is not already inside and add the new value if the word is already inside
                        New_Dict[term_id].append((doc, tf*idf))
                    else:
                        New_Dict[term_id] = [(doc, tf*idf)]
                else:
                    tf = tf_in 

                    if term_id in New_Dict.keys():  # add the word to the new dictioary if the word is not already inside and add the new value if the word is already inside
                        New_Dict[term_id].append((doc, tf*idf))
                    else:
                        New_Dict[term_id] = [(doc, tf*idf)]

Then using the same cose to compute the query's tdif, vectorization, cosine similarity and ranking, we could manage to rank our film with a better result respect to the second serching engine

### Query TF-IDF


In [59]:
def tf_idf_query(query):
    tf_idf = {}
    len_query = len(query)
    len_dict = len(os.listdir(path))
    DF = {}
    for word in query:# its a count funtion
        if word not in DF.keys():
            DF[word] = 1 
        else:
            DF[word]+=1
    
    for word in query:
        
        # TF = (Frequency of the word in the sentence) / (Total number of words in the sentence)
        count = query.count(word)
        freq = count/len_query
        tf = freq
        
        val_word = DF[word]
        idf = idf = np.log(len_dict/val_word)
        if word in Vocabulary:
            term_id = Vocabulary.index(word)
            if word in tf_idf.keys():
                tf_idf[str(term_id)]+= tf*idf
            else:
                tf_idf[str(term_id)] = tf*idf
    return tf_idf

In [60]:
tf_idf_query = tf_idf_query(query) 

### Vectorization, Cosine Similarity and Ranking

In [61]:
result_list = []
for i in range(len(result)):
    small_dict = {}
    for word in New_Dict.keys():
        for files in New_Dict[word]:
            if files[0] == result[i]:
                small_dict[word] = files[1]
    result_list.append(small_dict)

In [62]:
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [63]:
cos_result = {}
for i in range(len(result)):
    cosine = get_cosine(tf_idf_query, result_list[i])
    cos_result[result[i]] = cosine*100
    
cos_result = sorted(cos_result.items(), key=operator.itemgetter(1), reverse=True)

In [65]:
to_df = []
for movie in cos_result:
    element_in_movie = {}
    with open(path+movie[0]) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            if row[0] == 'Name':
                element_in_movie['Name'] = row[1]
            elif row[0] == 'Intro':
                element_in_movie['Intro'] = row[1]
            elif row[0] == 'Url':
                element_in_movie['Url'] = row[1]
        element_in_movie['Similarity'] = str(movie[1])+' %'
    to_df.append(element_in_movie)
df = pd.DataFrame(to_df)
df = df.reindex(('Name', 'Intro', 'Url', 'Similarity'), axis=1)
df.iloc[:10]

,Name,Intro,Url,Similarity
0,When in Rome,When in Rome is a 2010 American romantic comed...,https://en.wikipedia.org/wiki/When_in_Rome_(20...,6.835857966671183 %
1,Dear John,Dear John is a 2010 American romantic drama-wa...,https://en.wikipedia.org/wiki/Dear_John_(2010_...,5.704633376565299 %
2,I Love You Phillip Morris,I Love You Phillip Morris is a 2009 black come...,https://en.wikipedia.org/wiki/I_Love_You_Phill...,5.264481089373219 %
3,My One and Only,My One and Only is a 2009 comedy-drama film lo...,https://en.wikipedia.org/wiki/My_One_and_Only_...,4.689575381407523 %
4,Never Let Me Go,Never Let Me Go is a 2010 British dystopian ro...,https://en.wikipedia.org/wiki/Never_Let_Me_Go_...,4.209033222443061 %
5,Eat Pray Love,Eat Pray Love is a 2010 American biographical ...,https://en.wikipedia.org/wiki/Eat_Pray_Love,3.918970377109158 %
6,I Can Do Bad All by Myself,I Can Do Bad All by Myself is a 2009 romantic ...,https://en.wikipedia.org/wiki/I_Can_Do_Bad_All...,3.623062757513399 %
7,The Informers,The Informers is a 2008 American ensemble Holl...,https://en.wikipedia.org/wiki/The_Informers_(2...,3.2380426575731334 %
8,The Limits of Control,The Limits of Control is a 2009 American film ...,https://en.wikipedia.org/wiki/The_Limits_of_Co...,3.1846058945767446 %
9,Hurricane Season,Hurricane Season is a 2010 sports drama film d...,https://en.wikipedia.org/wiki/Hurricane_Season...,3.1793814369053632 %
